In [1]:
import torch
import dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2

In [2]:
model_path = '/opt/ml/upstage_OCR/code/saved/unet_focal/model.pt'
state_dict_path = '/opt/ml/upstage_OCR/code/saved/unet_focal/last.pt'

model = torch.load(model_path)
model.load_state_dict(torch.load(state_dict_path))

ann_path = '/opt/ml/upstage_OCR/Data set/annotations/general_00_10.json'
ann_path_val = '/opt/ml/upstage_OCR/Data set/annotations/general_46_60.json'
ocr_url = "http://118.222.179.32:30000/ocr/"
image_root = '/opt/ml/upstage_OCR/Data set/real data/general'
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
transform = A.Compose([
    A.Resize(512,512),
    ToTensorV2()
])

train_dataset = dataset.WifiDataset_segmentation(ann_path,ocr_url,image_root,transform=transform)
val_dataset = dataset.WifiDataset_segmentation(ann_path_val,ocr_url,image_root,transform=transform)

  0%|          | 0/14 [00:00<?, ?it/s]

loading annotations into memory...
Done (t=0.00s)
load images ...


  0%|          | 0/15 [00:00<?, ?it/s]

loading annotations into memory...
Done (t=0.00s)
load images ...


100%|██████████| 15/15 [00:09<00:00,  1.66it/s]


In [3]:
# collate_fn needs for batch
def collate_fn(batch):
    return tuple(zip(*batch))

train_dataloader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=4,
                                           shuffle=True,
                                           collate_fn=collate_fn)
val_dataloader = torch.utils.data.DataLoader(dataset=val_dataset, 
                                           batch_size=4,
                                           shuffle=True,
                                           collate_fn=collate_fn)

x,y = next(iter(train_dataloader))

In [9]:
import matplotlib.pyplot as plt
import torchvision
t = torchvision.transforms.Compose(
    [torchvision.transforms.ToPILImage()]
)

model.eval()
image = model(torch.stack(x).to(device))[0]
out = torch.argmax(image,dim=0)

plt.figure(figsize=(10,10))
plt.imshow(t(out*0.3))


plt.show()

RuntimeError: CUDA out of memory. Tried to allocate 256.00 MiB (GPU 0; 31.75 GiB total capacity; 30.36 GiB already allocated; 197.50 MiB free; 30.42 GiB reserved in total by PyTorch)

In [11]:
image[0]

tensor([[0.6104, 0.5448, 0.5177,  ..., 0.5186, 0.5394, 0.5987],
        [0.5466, 0.4456, 0.4305,  ..., 0.4332, 0.4446, 0.5250],
        [0.5155, 0.4400, 0.4282,  ..., 0.4373, 0.4399, 0.5060],
        ...,
        [0.6759, 0.6486, 0.6473,  ..., 0.6399, 0.6445, 0.6651],
        [0.6725, 0.6407, 0.6349,  ..., 0.6272, 0.6341, 0.6652],
        [0.6586, 0.6294, 0.6246,  ..., 0.6085, 0.6155, 0.6635]],
       device='cuda:0', grad_fn=<SelectBackward>)